# PageRank
#### Author : Alex Fender

In this notebook, we will show how to use multi-GPU features in cuGraph to compute the PageRank of each user in Twitter's dataset.

Please be aware that your system may be different, and you may need to modify the code or install packages to run the below examples. If you think you have found a bug or an error, please file an issue in [cuGraph](https://github.com/rapidsai/cugraph/issues)

This notebook was run on 2x NVIDIA Tesla 32GB V100 GPUs (connected with nvlink) using RAPIDS 0.9.0 and CUDA 10.0. It also works on 2x NVIDIA 32GB GV100 GPUs using RAPIDS 0.10.0 and CUDA 10.0

## Introduction
Pagerank is measure of the relative importance of a vertex based on the relative importance of its neighbors.  PageRank was invented by Google Inc. and is (was) used to rank its search results. PageRank uses the connectivity information of a graph to rank the importance of each vertex. See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

CuGraph's multi-GPU features leverage Dask. RAPIDS has other projects based on Dask such as dask-cudf and dask-cuda. These products will also be used in this example. Check out [RAPIDS.ai](https://rapids.ai/) to learn more about these technologies.

---

To compute the Pagerank with cuGraph we use:<br>

```python
cugraph.dask.pagerank.pagerank(edge_list, alpha=0.85, max_iter=30)
```
Parameters

*  *edge_list* : `dask_cudf.DataFrame`<br>
Contain the connectivity information as an edge list. Source 'src' and destination 'dst' columns must be of type 'int32'. Edge weights are not used for this algorithm. Indices must be in the range [0, V-1], where V is the global number of vertices. The input edge list should be provided in dask-cudf DataFrame with one partition per GPU.
*  *alpha* : `float`<br>
The damping factor alpha represents the probability to follow an outgoing edge, standard value is 0.85. Thus, 1.0-alpha is the probability to “teleport” to a random vertex. Alpha should be greater than 0.0 and strictly lower than 1.0.
* *max_iter* : `int`<br>
The maximum number of iterations before an answer is returned. If this value is lower or equal to 0 cuGraph will use the default value, which is 30. In this notebook, we will use 20 to compare against published results.<br>

Returns

* *PageRank* : `dask_cudf.DataFrame`<br>
Dask GPU DataFrame containing two columns of size V: the vertex identifiers and the corresponding PageRank values.

## Data
We will be analyzing 41.7 million user profiles and 1.47 billion social relations from the Twitter dataset.  The CSV file is 26GB and was collected in :<br>
*What is Twitter, a social network or a news media? Haewoon Kwak, Changhyun Lee, Hosung Park, and Sue Moon. 2010.*<br> 

---

The fastest way to obtain the dataset is to run :
```bash
sh ./get_data.sh
```

Please refer to the README for further information and more options on how to obtain this dataset.


## Multi-GPU PageRank with cuGraph
### Basic setup

In [ ]:
# Let's check out our hardware setup
!nvidia-smi

# GPUs should be connected with NVlink
!nvidia-smi nvlink --status

# For best performance, we can limit the number of available devices
# For this dataset, we can use only 2 Tesla V100(32GB) or 4 Tesla P100 (16GB).  We default to the first and second GPU, so please adjust the device IDs as necessary.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# List available devices
!echo Available GPUs: $CUDA_VISIBLE_DEVICES

In [ ]:
# Import needed libraries. We recommend using cugraph_dev env through conda
import time
from dask.distributed import Client, wait
import dask_cudf
from dask_cuda import LocalCUDACluster
import cugraph.dask.pagerank as dcg

### Setup multi-GPU and Dask

Before we get started, we need to setup a Dask local cluster of workers to execute our work and a client to coordinate and schedule work for that cluster. As we see below, we can initiate a `cluster` and `client` using only 2 lines of code.

In [ ]:
cluster = LocalCUDACluster(threads_per_worker=1)
client = Client(cluster)

### Get the data

The Twitter dataset is in our S3 bucket and zipped.  
1. We'll need to create a folder for our data in the `/data` folder
1. Download the zipped data into that folder from S3 (it will take some time as it it 6GB)
1. Decompress the zipped data for use (it will take some time as it it 26GB)

In [ ]:
import urllib.request
import os

data_dir = '../../../data/twitter/'
if not os.path.exists(data_dir):
    print('creating twitter data directory')
    os.system('mkdir ../../../data/twitter')

In [ ]:
# download the Twitter dataset
base_url = 'https://s3.us-east-2.amazonaws.com/rapidsai-data/cugraph/benchmark/'
fn = 'twitter-2010.csv'
comp = '.gz'
if not os.path.isfile(data_dir+fn):
    if not os.path.isfile(data_dir+fn+comp):
        print(f'Downloading {base_url+fn+comp} to {data_dir+fn+comp}')
        urllib.request.urlretrieve(base_url+fn+comp, data_dir+fn+comp)
    print(f'Decompressing {data_dir+fn+comp}...')
    os.system('gunzip '+data_dir+fn+comp)
    print(f'{data_dir+fn+comp} decompressed!')
else:
    print(f'Your data file, {data_dir+fn}, already exists')

### Read the data from disk
cuGraph depends on dask-cudf for data loading and the initial DataFrame creation. The CSV data file contains an edge list, which represents the connection of a vertex to another. The source to destination pairs is what is known as Coordinate Format (COO). In this test case, the data is just two columns. 

In [ ]:
# File path, assuming Notebook directory
input_data_path = data_dir+fn

# Helper function to set the reader chunk size to automatically get one partition per GPU  
chunksize = dcg.get_chunksize(input_data_path)

# Start timer
t_start = time.time()

# Multi-GPU CSV reader
e_list = dask_cudf.read_csv(input_data_path, chunksize = chunksize, delimiter=' ', names=['src', 'dst'], dtype=['int32', 'int32'])

# Wait for the lazy reader
tmp = wait(client.compute(e_list.to_delayed()))

# Print time
print(time.time()-t_start, "s")

### Call the Multi-GPU PageRank algorithm


In [ ]:
# Start timer
t_start = time.time()

# Get the pagerank scores
pr_ddf = dcg.pagerank(e_list, max_iter=20)

# Print time
print(time.time()-t_start, "s")

It was that easy! PageRank should only take a few seconds to run on this 26GB input with 2 Tesla V100 GPUs.<br>
Check out how it compares to published results in the [Annex](#annex_cell).

### Further analysis on the Pagerank result

We can now identify the most influent users in the network.<br>
Notice that the PageRank result can fit in one GPU. Hence, we can gather it in a regular `cudf.DataFrame`. We will then sort by PageRank value and print the *Top 3*.

In [ ]:
# Start timer
t_start = time.time()

# Dask Data Frame to regular cuDF Data Frame 
pr_df = pr_ddf.compute()

# Sort, descending order
pr_sorted_df = pr_df.sort_values('pagerank',ascending=False)

# Print time
print(time.time()-t_start, "s")

# Print the Top 3
print(pr_sorted_df.head(3))

We can now use the [map](https://s3.us-east-2.amazonaws.com/rapidsai-data/cugraph/benchmark/twitter-2010-ids.csv.gz) to convert Vertex ID into to Twitter's numeric ID. The user name can also be retrieved using the [TwitterID](https://tweeterid.com/) web app.<br>
The table below shows more information on our *Top 3*. Notice that this ranking is much better at capturing network influence compared the number of followers for instance. Further analysis of this dataset was published [here](https://doi.org/10.1145/1772690.1772751).

| Vertex ID	| Twitter ID	| User name	| Description |
| --------- |  ---------   | --------   |   ----------  |
| 21513299	| 813286	| barackobama	| US President (2009-2017) |
| 23933989	| 14224719	| 10DowningStreet | UK Prime Minister office |
| 23933986	| 15131310	| WholeFoods	| Food store from Austin |



### Close the multi-GPU environment

In [ ]:
client.close()
cluster.close()

## Annex
<a id='annex_cell'></a>
An experiment comparing various porducts for this workflow was published in *GraphX: Graph Processing in a Distributed Dataflow Framework,OSDI, 2014*. They used 16 m2.4xlarge worker nodes on Amazon EC2. There was a total of 128 CPU cores and 1TB of memory in this 2014 setup.

![twitter-2010-spark.png](twitter-2010-spark.png)

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___